# Custom Models and Training with TensorFlow

In [13]:
import tensorflow as tf
import numpy as np

## Using TensorFlow like NumPy

TensorFlow revolves around *tensors*, similar to NumPy `ndarray`: usually a multidimensional array, but it can also hold a scalar (simple value, such as `42`).

### Tensors and Operations

Create tensor with `tf.constant()`

In [2]:
tf.constant([[1.,2.,3.], [4.,5.,6.]]) # matrix

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
tf.constant(42) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

like `ndarray`, `tf.Tensor` has a shape and a data type (`dtype`)

In [5]:
t = tf.constant([[1.,2.,3.], [4.,5.,6.]])
t.shape

TensorShape([2, 3])

In [6]:
t.dtype

tf.float32

Indexing works like NumPy

In [7]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [8]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

Tensor operations

In [9]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [10]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [11]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

`@` is equivalent to calling `tf.matmul()` (matrix multiplication)

#### Keras' Low-Level API

Keras API has its own low-level API, `keras.backend`. Includes functions like `square()`, `exp()`, and `sqrt()`. In tf.keras, these functions just call corresponding TensorFlow operations. If you want to write code that will be portable to other Keras implementations, you should use these Keras functions. However, they only cover a subset of all the functions available in TensorFlow

In [12]:
from tensorflow import keras
import keras.backend as K
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

### Tensors and NumPy

You can create a tensor from a NumPy array, and vice versa. You can apply TensorFlow operations to NumPy arrays and NumPy operations to tensors.

In [14]:
a = np.array([2.,4.,5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [15]:
t.numpy() # or np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [16]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [17]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

Note NumPy uses 64-bit precision by default, while TensorFlow uses 32-bit. When you create a tensor from a NumPy array, make sure to set it to `dtype=tf.float32`

### Type Conversions

TensorFlow does not perform any type conversions automatically

In [18]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

In [19]:
tf.constant(2.) + tf.constant(40., dtype=tf.float64)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]

### Variables

`tf.Tensor` values are immutable. Use tf.Variable for mutable variables

In [21]:
v = tf.Variable([[1.,2.,3.,], [4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

`tf.Variable` behaves same way as `tf.Tensor` (same operations, with NumPy as well, picky with types, etc) but can also be modified in place using the `assign()` method (or `assign_add()` or `assign_sub()` which increment or decrement the variable by the given value). Can also modify cells (or slices) by using the cell's (or slice's) `assign()` method or `scatter_update()` or `scatter_nd_update()` (direct item assignment will not work)

In [22]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [23]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [25]:
v.scatter_nd_update(indices=[[0,0], [1,2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   6.],
       [  8.,  10., 200.]], dtype=float32)>

### Other Data Structures

- Sparse tensors (`tf.SparseTensor`)
- Tensor arrays (`tf.TensorArray`)
- Ragged tensors (`tf.RaggedTensor`)
- String tensors
- Sets
- Queues